# Introduction à Spark Core

Dans ce notebook, nous allons découvrir pas à pas l'API Spark Core et les RDD (pour _Resilient Distributed Dataset_).

N'hésitez pas à aller voir la documentation de Spark.
 * Scaladoc de l'API Spark : https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html

## Import des dépendances Spark
Le noyau [Almond](https://almond.sh/) va nous permettre de récupérer les dépendances Spark.

⚠️ Elles n'en ont pas l'air, mais les deux lignes ci-dessous peuvent potentiellement importer un grand nombre de dépendances. À chaque fois qu'une dépendance est en cours d'import ou est importée, une ligne est ajoutée dans la partie output (en fond rose). Pour éviter d'avoir trop de lignes affichées, cliquez droit sur la partie output et sélection "Enable Scrolling for Outputs". L'import des dépendances est terminé lorsqu'un numéro apparaît entre crochets, à la place d'une étoile, à gauche dans les lignes ci-dessous.

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

## Initialisation du contexte / de la session Spark

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("Sales Analysis")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

### Nous allons maintenant récupérer le SparkContext

Les lignes au-dessus permettent de récupérer ce qui s'appelle un SparkSession. Un SparkSession représente la configuration et le runtime utilisé par SparkSQL. Plus exactement, il s'agit un NotebookSparkSession. Il s'agit d'un wrapper mise en place par le noyau Almond, afin de fournir notamment des barres de progression pour les différents calculs que vous allez effectuer. À partir d'un SparkSession, il est possible de récupérer le SparkContext.

Il est à noter qu'habituellement, il est possible de créer un SparkContext directement sans passer par un SparkSession.

In [ ]:
// Récupération du SparkContext
val sparkContext = spark.sparkContext

## Lecture d'un fichier avec Spark Core

Nous allons récupérer le fichier `orders.csv` et réaliser des analyses sur ce fichier.

Commençons par afficher un extrait de son contenu.

In [ ]:
shell("cat orders.csv", limit=10)

Nous allons utiliser Spark pour récupérer le contenue de fichier.

Dans la cellule ci-dessous, vous allez utiliser la méthode `.textFile()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/SparkContext.html#textFile(path:String,minPartitions:Int):org.apache.spark.rdd.RDD[String])) sur le `sparkContext` pour récupérer le contenu du fichier `orders.csv`, situé dans le même répertoire que ce notebook.

La méthode `.textFile()` va récupérer le fichier, dont le nom est passé en paramètre. Comme, on demande de voir le fichier comme un fichier texte (`text` dans le nom de la méthode), il sera découpé ligne par ligne par cette méthode. Le résultat est un `RDD[String]`, où il faut voir le type `RDD` comme une collection. Le résultat est donc vu comme une collection de lignes, provenant du fichier `orders.csv`.

In [ ]:
val rawData: RDD[String] = ???

rawData.showHTML(title="Extrait de rawData", limit=10, truncate=120)

**Ce qu'il faut voir**

Nous avons réussi à charger le fichier CSV avec Spark et nous avons pu en afficher une partie. Cependant, nous pouvons remarquer que la première ligne, qui représente les en-têtes de colonne, devrait être retirée du dataset pour pouvoir être exploitable. C'est que nous allons faire dans la cellule ci-dessous.

Si vous allez voir dans la Spark UI, vous verrez qu'un job apparaît. Ce job est appelé `showHTML`. Nous pouvons voir son nom un peu plus dans ce notebook, juste au-dessus de la barre de progression.

In [ ]:
// Récupération de la première ligne (en-têtes)
val header: String = rawData.first()

// Récupération des lignes du dataset, sauf celle contenant les en-têtes de colonne
// et séparation des colonnes
val data: RDD[String] =
  rawData
    .filter(line => line != header)

data.showHTML(title="Extrait de data", limit=10, truncate=120)

**Ce qu'il faut voir**

Cette fois, nous avons 2 jobs qui ont été lancés : un job pour `first()` et un job pour `showHTML`. Cela se voit aussi au niveau des barres de progression, mais aussi dans la Spark UI. Dans le cadre du second job, dans la Spark UI, nous pouvons voir qu'il est composé de deux opérations : une pour récupérer le contenu du fichier et une autre pour filtrer l'en-tête des colonnes.

## Première requête : nombre total de ventes

Maintenant que nous avons chargé un fichier, nous pouvoir commencer à y effectuer des traitements.

Dans la cellule ci-dessous, utilisez la méthodes `.count()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html#count():Long)) sur le RDD `data` afin d'avoir le nombre total de ventes contenu dans le fichier.

In [ ]:
val totalSales: Long = ???

s"Nombre total de ventes: $totalSales"

**Ce qu'il faut voir**

Une différence subtile apparaît dans la barre de progression : nous voyons l'indication `2 / 2`. Ceci signifie que 2 tâches ont été utilisées pour exécuter le job. Une tâche représente une opération effectuée par un exécuteur sur une partition.

Jusque-là, seule une tâche avait été utilisée. Dans la mesure où nous avions à afficher seulement une partie des données, seule une seule partition était exploitée.

Dans le cas du `count`, nous avons besoin d'accéder à toutes les partitions utilisées par le contenu du fichier `orders.csv`. Or ce fichier est éparpillé sur 2 partitions.

L'expression ci-dessous va nous en convaincre.

In [ ]:
data.getNumPartitions

## Conversion

Avoir un RDD contenant les lignes d'un fichier sous forme de chaîne de caractères, ce n'est pas très pratique. Il nous faut convertir ces lignes dans une structure plus aisée à exploiter.

Nous allons créer la structure `Order`, qui va nous permettre de convertir les lignes dans un format plus simple à manipuler.

In [ ]:
import java.time._
import java.time.format._

case class Order(
  id:        String,
  clientId:  String,
  timestamp: LocalDateTime,
  product:   String,
  price:     Double
)

def toLocalDateTime(field: String): LocalDateTime =
  LocalDateTime.parse(
    field,
    DateTimeFormatter.ISO_LOCAL_DATE_TIME
  )

Nous allons maintenant convertir notre `RDD[String]` en `RDD[Order]`.

Pour cela, nous allons utiliser la fonction `.map()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html#count():Long)) sur le RDD `data`. `map` est une opération qui prend en paramètre une fonction et qui applique cette fonction sur chaque élément contenu dans le RDD.

Mais avant cela, nous allons définir la fonction qui va permettre de convertir une ligne est `Order`.

Note:
 * L'accès à un case `i` dans un tableau `a` (Array) se fait de la manière suivante : `a(i)`
 * Pour convertir une chaîne en LocalDateTime, nous allons utiliser la fonction `toLocalDateTime` définie plus haut
 * Pour convertir une chaîne une chaîne `s` en Double, il suffit de faire : `s.toDouble`

In [ ]:
def lineToOrder(line: String): Order = {
  val fields = line.split(",")
  Order(
    id = fields(0),
    clientId = fields(1),
    timestamp = toLocalDateTime(fields(2)),
    product = fields(3),
    price = fields(4).toDouble,
  )
}

Dans la cellule ci-dessous, nous allons utiliser la méthode `.map()` sur le RDD `data`.

Note : en Scala, pour représenter une fonction (ou _lambda expression_) qui prend un paramètre `s` de type String et applique une fonction `f` dessus, il suffit d'écrire : `(s: String) => f(s)`.

In [ ]:
val orders: RDD[Order] = ???

orders.showHTML(title="Extrait de orders", limit=10, truncate=120)

**Ce qu'il faut voir**

L'opération `.map()` permet d'appliquer une transformation à chaque ligne rencontrée dans le contexte du RDD. Nous n'avons plus un `RDD[String]`, mais un `RDD[Order]`. Une telle structure sera plus exploitable avec (à nouveau) l'opération `.map()`, mais aussi avec d'autres opérations applicables sur les RDD.

## Trouvez le produit le plus vendu (ID du produit et quantité totale vendue)

Trouver le produit le plus vendu va nécessiter :
 1. d'organiser les données par produit
 2. de compter ensuite le nombre de lignes pour chacun de ces produits
 3. de trier les produits par rapport à ce décompte
 4. de récupérer le produit avec le grand décompte.

### Organiser par produit
Pour la première étape, nous allons utiliser la méthode `.keyBy()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html#keyBy[K](f:T=%3EK):org.apache.spark.rdd.RDD[(K,T)])) sur le RDD `orders`. Cette méthode prend en paramètre une fonction qui va indiquer la valeur utilisée pour la clé à partir d'un order (ici, ce sera le champ `product`). De cette méthode, il en sort un RDD de couple _nom du produit / order_. Ce type de RDD s'appelle un PairRDD et des méthodes spécifiques s'appliquent sur les RDD de cette catégorie.

In [ ]:
val ordersByProduct: RDD[(String, Order)] = ???

ordersByProduct.showHTML(limit=10, truncate=120)

**Ce qu'il faut voir**

Chaque ligne se voit associer un nom de produit correspondant. Dans sa façon de le représenter, Spark utilise des tuples Scala, avec la notation `(a, b)`.

### Compter par produit (1)
Nous allons compter le nombre de commandes par produit. Pour cela, nous allons d'abord associer à chaque ligne la valeur `1` pour pouvoir ensuite faire la somme de ces `1`.

Pour cela, nous allons réutiliser l'opération `.map()` pour convertir le `RDD[(String, Order)]` en `RDD[(String, Int)]`, où `Int` apparaît, car nous allons remplacer Order par al valeur `1`.

Note : comme pour l'opération `.map()`, nous partons d'un `RDD[(String, Order)]`, la fonction dans `map` aura une écriture un peu différente. On utilisera ce format `{ case (a, b) => /* do something with a and b */ }`.

In [ ]:
val productAndOne: RDD[(String, Int)] = ???

productAndOne.showHTML(truncate=120)

### Compter par produit (2)
Nous allons à présent compter par produit le nombre de `1`. Dans ce cadre, nous allons utiliser la méthode `.reduceByKey()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html#reduceByKey(func:(V,V)=%3EV):org.apache.spark.rdd.RDD[(K,V)])).

Cette méthode récupère les éléments un par un du `RDD[(String, Int)]`. Pour chaque élément, la partie String (nom du produit) va servir de clé, sachant qu'un résultat est généré par clé. La partie Int (qui vaut `1`) est utilisé dans la fonction passée en paramètre de `.reduceByKey()`.

Ainsi, lorsqu'une nouvelle donnée est récupérée du RDD, sa valeur est passée en paramètre de la fonction dans `reduceByKey`. La valeur est alors agrégée avec un résultat intermédiaire. Le résultat de cette agrégation devient le nouveau résultat intermédiaire qui est utilisé pour l'élément suivant du RDD (pour la même clé).

La fonction passée en paramètre de `reduceByKey` est fonction qui ressemble à `(a, b) => f(a, b)`, sachant que `a`, `b` et le résultat de l'agrégation de `a` et de `b` doivent tous avoir le même type.

In [ ]:
val productCount: RDD[(String, Int)] = ???

productCount.showHTML(truncate=120)

**Ce qu'il faut voir**

Nous voyons à présent 3 barres de progression pour un seul job (showHTML).

Sachant que nous avons besoin maintenant de toutes les données pour compter, les opérations que nous avons mises en place depuis la lecture du fichier jusqu'au `map` vont être exécutées sur les deux partitions contenant les données du fichier. Ce qui donne 2 tâches à ce niveau. Puis, le `reduceByKey` nécessite une étape préalable consistant à mettre les données ayant la même clé au niveau de la même unité de travail, au niveau du même exécuteur. Ceci se traduit par un échange de données (_shuffle_) et donc la mise place d'un nouveau _stage_, un pour chaque partition. C'est la raison pour laquelle nous avons 2 barres de progression de plus.

Dans Spark UI, nous pouvons voir apparaître 2 nouveaux jobs showHTML. Sachant que pour le premier, l'ensemble des opérations sont faites au complet. Et pour le second job, certains _stages_ et tâches sont passées, car déjà effectuées dans le cadre du premier job.

Note : lorsqu'il y a du _shuffle_, Spark doit générer les données avant de pouvoir les échanger. Cette opération tend à remplir la mémoire et les disques (au travers de la génération de fichiers) associés aux exécuteurs. C'est aussi ce qui fait que le _shuffle_ est coûteux. Les fichiers générés sont conservés jusqu'à ce que le RDD associé ne soit plus utilisé.

### Tri par décompte
Nous allons maintenant trier les produits selon leur quantité vendue. Pour cela, nous utiliserons la fonction `.sortBy()` ([lien](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html#sortBy[K](f:T=%3EK,ascending:Boolean,numPartitions:Int)(implicitord:Ordering[K],implicitctag:scala.reflect.ClassTag[K]):org.apache.spark.rdd.RDD[T])). Cette fonction prend en paramètre une fonction qui à partir d'un élément du RDD extrait une valeur qui va servir à trier ces éléments entre eux. Un deuxième paramètre (_ascending_) optionel indique l'ordre dans lequel sont triés les éléments.

In [ ]:
val productSortedByCount: RDD[(String, Int)] = ???

productSortedByCount.showHTML(truncate=120)